## Texual Inversion 
count # of trainable params

In [1]:
def count_params(model, verbose=True):
    total_params = sum(p.numel() for p in model.parameters())
    if verbose:
        print(f"{model.__class__.__name__} has {total_params*1.e-6:.2f} M params.")
    return total_params


In [11]:
from transformers import CLIPTextConfig

config = CLIPTextConfig.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="text_encoder")
config.hidden_size

768